In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.mixture import GaussianMixture
import re
from collections import defaultdict
from bud_lib import *

tqdm.pandas()
nlp = Polish()
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

2021-12-27 00:32:38.064051: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-27 00:32:38.064119: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-27 00:32:41.329525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-27 00:32:41.329586: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-27 00:32:41.329606: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mlowanie): /proc/driver/nvidia/version does not exist
2021-12-27 00:32:41.329841: I tensorflow/core/platform/cpu_feature_g

In [3]:
data = pd.read_csv('budzet_dataset.csv')
data = data.dropna() #just in case
data = data.head(10)
sentences = data["sentence"].values

In [4]:
data["tok"] = data["sentence"].progress_apply( lambda x: np.array([token.text for token in nlp(x)]))
all_tokens = np.concatenate(data["tok"].values)
del data["tok"] # no need to store tokenized sentences

#pick 300 most common tokens
unique, counts = np.unique(all_tokens, return_counts=True)
tokens_cnt = dict(zip(unique, counts))
tokens_cnt = dict(reversed(sorted(tokens_cnt.items(), key=lambda item: item[1])))
tokens_cnt = list(tokens_cnt.items())[:300]
topics = np.array([i for i in tokens_cnt])
topic_words = np.array([i[0] for i in topics])

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
embeddings = embed_list(topic_words)

  0%|          | 0/52 [00:00<?, ?it/s]

In [6]:
gm = GaussianMixture(n_components=20, random_state=0).fit(embeddings)
prediction = gm.predict(embeddings)

In [7]:
name_in_group =  dict(zip(topic_words.tolist(), prediction))

res = defaultdict(list)
for key, val in sorted(name_in_group.items()):
    res[val].append(key)

for i in res:
    print(res[i])
    print("\n")


['architektura']


['atrakcyjność', 'koncentracja']


['bohater', 'dorosły', 'senior', 'wieżowiec']


['bujny', 'niski', 'skwer']


['chodnik', 'ogród', 'park', 'pergola']


['doposażyć', 'ligać', 'montaż', 'nasadzić']


['dziecko', 'dąbek']


['dzielnicowy', 'towarzyski']


['elementy', 'mebel', 'ozdobny', 'ławka']


['festyn']


['kontener', 'podwórek']


['kurs', 'lekcja', 'taniec', 'wycieczka']


['morski']


['naukowy', 'roślina', 'źródlany']


['odnowić', 'poprawa', 'rozwijać', 'zaprojektować']


['oświetleniowy', 'radarowy', 'wyświetlacz']


['pamięć', 'prędkość']


['piłkarski']


['pracownia', 'słup', 'teatr', 'warsztaty']


['siłownia']


